## Лекция 2  BM5    

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

# Далее идет работа, которую я делал до того, как пришлось пропускать занятия из-за защиты курсовой. 
Я посмотрел ее и понял, что не понял задание, когда делал. Поэтому пришлось переделывать, но удалять ячейки я не стал, потому что опирался на них в новом коде. Код, который я буду имплементировать в проект дан после, отделен еще одной чертой.
***

## Функция ранжирования bm25

### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- **ГОТОВО** проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- **ГОТОВО** функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


 ## Сделать матрицу для базы запросов
1. прочесть файл
2. получить файл в пандас
3. разделить матрицу пандас в 70/30

In [ ]:
import pandas as pd
# обозначение датафрейма со всеми запросами
df = pd.read_csv("queries_base.tsv", sep="\t")
df.columns=["text", "connection", "theme"]
df.head()

In [ ]:
# деление основного датафрейма со всеми запросами
# на тренировочный и тестовый датафреймы
perc70 = int(int(df.shape[0])/10*7)
df_train = df.iloc[:perc70]
df_test = df.iloc[perc70:]
df_test.shape, df_train.shape

## Сделать функцию перевода 
1. сделать функцию перевода строки, чтобы переводить запрос
2. сделать функцию перевода матрицы пандас

In [ ]:
# перевод тестового и тренировочного датафреймов в TF-IDF матрицы
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
v = TfidfVectorizer()
df_train_tfidf = v.fit_transform(df_train['text'].values.astype('U'))
df_test_tfidf = v.transform(df_test['text'].values.astype('U'))
df_train_tfidf

In [ ]:
# обозначение векторайзера и перевод строки 
# маска пожалуйста 
query = [0]
query[0] = str((input('ваш запрос: ')))
print(query)
query_tfidf = v.transform(query)
print(query_tfidf)
type(query_tfidf)

In [ ]:
answer = df_train_tfidf.dot(query_tfidf.T).toarray()

In [ ]:
answer_sorted = np.sort(answer, axis=0)
answer_sorted = answer_sorted[::-1]
answer_sorted[:10]

In [ ]:
i = 0
answer_sorted[i][0]
while answer_sorted[i][0] != 0:
    

In [ ]:
df_answers = pd.DataFrame()
i = 0
while answer_sorted[i][0] != 0:
    doc_num = np.where(answer == answer_sorted[i][0])[0][0]
    df_answers = df_answers.append(df.loc[doc_num])
    i += 1
df_answers

### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [ ]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75

def bottom(dl, avgdl):
    return k * ((1-b) + b * (float(dl)/float(avgdl)) )

def bm25(n, f, qf, r, N, dl, avdl) -> float:
    bottom_coef = bottom(dl, avdl)
    idf_component = log( ( (r + 0.5) / (0 - r + 0.5) ) / ( (n - r + 0.5) / (N - n - R + r + 0.5)) )
    return 


****
# Здесь заканчивается кусок кода для опоры, начинается новый код.

In [276]:
import pandas as pd
import numpy as np

import string # для .punctuatuion 
import math # bm25
from collections import OrderedDict # для сортировки
from collections import Counter # для матрицы бм25

from razdel import tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

### Препроцессинг

У меня очень мало времени на создание финального проекта, поэтому для удобного использования данных в будущем, я буду сразу пропускать их через базовый препроцессинг, который потом можно будет улучшить на базе третьего задания, но его пока не сделал. 


In [277]:
morph = MorphAnalyzer()
stop = set(stopwords.words('russian'))

def prep(unprep):
    unprep = str(unprep)
    unprep = unprep.lower()
    unprep = unprep.replace('\n', ' ') #.punctuation не прочтет
    unprep = unprep.translate(str.maketrans('', '', string.punctuation))# https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    unprep_tokenized = list(tokenize(unprep))
    lemmas = [morph.parse(i.text)[0].normal_form for i in unprep_tokenized]
    words = [i for i in lemmas if i not in stop]
    return ' '.join(words)

### Создание датафреймов с данными и получение корпуса

In [278]:
# теперь читать можно сразу из экселя
answers_df = pd.read_excel("answers_base.xlsx")
questions_df = pd.read_excel("queries_base.xlsx")

In [279]:
# названия столбцов неудобно вызывать в pandas, переименую
# тематика не понадобится
answers_df.rename(columns={'Номер связки': 'connection', 'Текст вопросов': 'text'}, inplace=True)
questions_df.rename(columns={'Номер связки\n': 'connection', 'Текст вопроса': 'text'}, inplace=True)

In [280]:
# делим на тест и трейн красиво, не слайсом
train_df, test_df = train_test_split(questions_df, test_size=0.3)
# объединяем все тренировочные тексты вопросов
qa_df = pd.concat([answers_df, train_df])

/Users/egor/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [281]:
qa_df = qa_df.drop(columns = ['Unnamed: 3','Unnamed: 4'])
qa_df

,connection,text,Текст ответа,Тематика,Тематика
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,NaN
1,78.0,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,NaN
2,326.0,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,NaN
3,327.0,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,NaN
4,328.0,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,NaN
...,...,...,...,...,...
550,37.0,"Добрый день, устроился на работу в город усинс...",NaN,NaN,"ОГРАНИЧЕНИЯ, ПРОПУСКНАЯ СИСТЕМА И ПЕРЕМЕЩЕНИЕ ..."
887,308.0,">Добрый день, \n>вернулись из Турции ( туристи...",NaN,NaN,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА..."
430,37.0,"Здравствуйте, подскажите пожалуйста, если я пр...",NaN,NaN,"ОГРАНИЧЕНИЯ, ПРОПУСКНАЯ СИСТЕМА И ПЕРЕМЕЩЕНИЕ ..."
1632,1.0,"Скажите, пожалуйста, когда можно считать каран...",NaN,NaN,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n"


In [282]:
# составляю корпус документов, как в примере
corpus = []
for question in qa_df['text']:
    corpus.append(prep(question))

In [283]:
# делаю матричку для трейна
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(corpus)
X_train.shape

(1652, 7410)

In [284]:
# сразу напишу перевод текста в вектор для запроса 
def txt_to_tfidf(txt, vectorizer):
    txt = prep(txt)
    return vectorizer.transform([txt]).toarray()

### Теперь сделаю тестовый запрос и отранжирую ответ

In [285]:
# запросом будет текст из тестовой выборки под индексом, допустим, 1
txt = test_df.text.values[1]
txt

'Если я вернулся из-за границы и должен сдать тест на коронавирус в течении трех дней, то в какой срок я должен загрузить результаты на сайт госуслуг?\xa0\n'

In [286]:
# преобразуем запрос в вектор
test_query = txt_to_tfidf(txt, vectorizer)
test_query

array([[0., 0., 0., ..., 0., 0., 0.]])

In [287]:
# транспонируем вектор, чтобы его можно было корректно умножить
# на полученную ранее матрицу трейна.
# получатся значения близости.
test_answer = X_train.dot(test_query.T)
# убедимся, что все сделали верно
len(test_answer.T[0])

1652

### Посмотрим самые популярные тексты и к каким ответам они привязаны

In [288]:
# совместим ответы с их номерами связок = list[ tuple(V, CONN), … ]
test_answer_conn = list(zip(test_answer.T[0], qa_df['connection'].values))

In [290]:
# отсортируем список кортежей по значению близости по возрастанию
sorted_tac = sorted(test_answer_conn, key=lambda tup_k: tup_k[0], reverse=True)
sorted_tac[:10]

[(0.452095129379077, 308.0),
 (0.43830992732692803, 6.0),
 (0.37465503294320923, 308.0),
 (0.35575890128910914, 308.0),
 (0.34420283138094426, 308.0),
 (0.34078527812031256, 308.0),
 (0.34057574605392016, 308.0),
 (0.3343846283059541, 308.0),
 (0.3307979114856545, 308.0),
 (0.3299929697198379, 308.0)]

Самый популярный ответ - 308 связка, отлично!

# BM25

### Что нужно для этой части:
1. Обратно индексированный словарь
2. Векторизатор запроса
3. Векторизатор трейна 
4. Создание матрицы по полученным функциям


### Словарь обратной индексации

In [335]:
def dict_inversed(x_train):
    x_train = x_train.toarray()
    # это будет словарь вида dict('feature_name':[sum,  ind_1,ind_2, … ind_n,feature_index])
    output_dict = {}
    # маплю, получаю список названий фичей
    f_names = vectorizer.get_feature_names()
    for f_index, f_name in enumerate(f_names):
        # сумма единичек по горизонтали по фичам 
        output_dict[f_name] = [int(sum(x_train[:, f_index]))]
        for index, d_index in enumerate(x_train[:, f_index].tolist()):
            # проход по вертикали - по каждому документу. если встречается, то записываем
            if d_index != 0:
                output_dict[f_name].append(d_index)
        # последний параметр в списке для ключа-фичи - индекс фичи
        output_dict[f_name].append(f_index)
    return output_dict

### Векторизатор запроса

In [341]:
def bm25_vectorizer_small(txt, output_dict):
    # создаем массив-строку из нулей длиной равной количеству фич-слов
    new_v = np.zeros((1, len(output_dict)))
    txt = prep(txt)
    for word in txt.split(' '):
        if word in output_dict.keys():
            # заполним нужные места в новом векторе
            new_v[0, output_dict[word][-1]] = 1
    return new_v

### Векторизатор тренировочных данных

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [337]:
def bm25_vectorizer_large(tf_val, ld, corpus, nq):
    k = 2.0
    b = 0.75
    lcorp = len(corpus)
    avgdl = sum([len(i.split(" ")) for i in corpus]) / lcorp
    IDF = np.log((lcorp-nq+0.5) / (nq+0.5))
    TF = (tf_val * (k+1)) / (tf_val + k * (1-b+b*(ld / avgdl)))
    return TF * IDF

### Реализация матрицы

In [339]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(corpus)
output_dict = dict_inversed(X_train)
bm25_matrix = np.zeros((len(corpus), len(output_dict)))

for txt_index, txt in enumerate(corpus):
    words = txt.split(' ')
    # посчитаем параметры для BM25
    tf_val = collections.Counter(words)
    ld = len(words)
    for word in words:
        if word in output_dict.keys():
            bm25_matrix[txt_index, output_dict[word][-1]] = bm25_vectorizer_large(tf_val[word], ld, corpus, len(output_dict[word]) - 1)
        else:
            continue

### Сделаем запрос по тестовой выборке, как для TF-IDF

In [342]:
txt = test_df.text.values[1]
test_query = bm25_vectorizer_small(txt, output_dict)

In [344]:
test_answer = bm25_matrix.dot(test_query.T)
test_answer.T[0]

array([ 4.89236492,  5.42282486,  0.        , ..., -0.42726096,
        0.        ,  0.        ])

In [345]:
# совместим ответы с их номерами связок = list[ tuple(V, CONN), … ]
test_answer_conn = list(zip(test_answer.T[0], qa_df['connection'].values))

In [346]:
# отсортируем список кортежей по значению близости по возрастанию
sorted_tac = sorted(test_answer_conn, key=lambda tup_k: tup_k[0], reverse=True)
sorted_tac[:10]

[(21.422809684123482, 308.0),
 (20.46379378745481, 6.0),
 (19.830440142190596, 308.0),
 (17.72524128773616, 308.0),
 (17.319884634754775, 308.0),
 (16.535023339587845, 308.0),
 (16.37148241245886, 308.0),
 (16.260036869785537, 308.0),
 (16.170457492321397, 308.0),
 (15.485265384978952, 308.0)]

Самый популярный ответ - 308 связка, отлично! Результат по метрикам одинаковый!